In [44]:
import numpy as np
#QUBIT base info
#pure states for now, mixed state add later
#multiparticle states can come later too.


class qubit:
    #Base qubit: has internal (private) methods for executing arbitrary rotations. Density matrix is the truest register for internal state.
    #density matrix goes as: first column [0][0] [0][1], second column [1][0] [1][1]
        #so each list in the wider array is a column
    rho = np.array([[1,0],[0,0]], dtype = complex)

    #paulis. px = pauli x, etc.
    i2 = np.array([[1,0],[0,1]])
    px = np.array([[0,1],[1,0]])
    py = np.array([[0,1j],[-1j,0]])
    pz = np.array([[1,0],[0,-1]])

    # avoid using pVec. it is treated as a (1, 3, 2, 2) tensor instead of a (1, 3) vector
    pVec = np.array([[px,py,pz]])
    
    
    # initialize in z down state
    def __init__(self):
        self.rho = [[1+0j,0+0j],[0+0j,0+0j]]

    # set density state to one reflecting the given z basis [[up, down]] input
    # \ket{state}\bra{state}, outer product
    def setState(self, zBasisVec):
        self.rho = np.outer(zBasisVec, zBasisVec)
    
    #set density state to blcoh vector equivalent
    #pauli decomposition
    def setStateVector(self, blochVec):
        blochVec /= np.linalg.vector_norm(blochVec)
        self.rho = 0.5*(self.i2 + blochVec[0][0]*self.px + blochVec[0][1]*self.py + blochVec[0][2]*self.pz)

    # set density state to matrix input
    def setStateDensity(self, matrix):
        self.rho = matrix

    # return density state matrix
    def state(self):
        return( np.around(self.rho , 5))
        
    # normalize trace to one and return dmatrix copy
    def stateNormalized(self):
        a = np.trace(self.rho)
        rhoNormalized = np.zeros((2,2), dtype = complex)
        
        for i in range(len(self.rho)):
            for j in range(len(self.rho[0])):
                rhoNormalized[i][j] = self.rho[i][j] / a
        return( np.around(rhoNormalized, 5))

    # pure if trace( rho^2 ) == 1
    #note: python 3.5 "@" operator does matrix multiplication for base arrays, np.matmul does the same, np.dot works fine for 2d but not for higher dimension
    def isPure(self):
        return (1 == np.around( np.trace(np.dot(self.rho, self.rho)) , 5))
        
    # expected value of a hermitian operator "measurement" on rho: Tr(rho * operator)
    def expectedValue(self, operator):
        return np.around( np.trace(np.dot(self.rho, operator)) , 5)


    def blochVector(self):
        b = np.zeros((1, 3), dtype = complex)
        b[0][0] = self.expectedValue(self.px)
        b[0][1] = self.expectedValue(self.py)
        b[0][2] = self.expectedValue(self.pz)
        return np.around(b, 5)

    # rotate around 3d axis (unit) vector by theta. 
    # normalize nVector, then construct rotation matrix from paulis: U_{n, theta} = I\cos(\theta) + i\sin(\theta) * (n \cdot \Vec{\sigma})
    # multiply rot^\dag * rho * rot
    # wikipedia Bloch_sphere > rotation operators 
    def rotate(self, nVec, theta): #theta in radians, nVector must be in the format [[n_x, n_y, n_z]] with two brackets (so n by 1 matrix)
        nVec /= np.linalg.norm(nVec)
        rot = np.array(self.i2) * np.cos(0.5*theta) - (0+1j) * np.sin(0.5*theta) * ( nVec[0][0]*self.px + nVec[0][1]*self.py + nVec[0][2]*self.pz )
        self.rho = np.dot( np.dot( np.conjugate( np.transpose(rot) ), self.rho), rot)




In [ ]:
# operates in terms of pulses
# larmor (precession) frequency omega, rabi frequency o_rabi
# methods wishlist:
    # omega and o_rabi methods
    # "hamiltonian" based on time functions or templates. omega and o_rabi based on time + B field signals / phase variance
        # input data + timestamps of B field signals, output evolution rate
    # setting T1,T2 or solving from lindbladian oeprator
    
class qubitsim(qubit):
    
    # counterclockwise qubit individual precession frequency (larmor)
    omega = 0

    # counterclockwise rotating frame of reference relative to lab frame
    omega_rof = 0

    # longitudinal relaxation rate. None means no decay (equiv to infinite)
    t1 = None

    # transverse relaxation rate. t2 \leq t1
    t2 = None
    

    # omega = (E_1-E_0) / hbar
    def setOmega(self, o):
        self.omega = o

    def setT1(self, t1):
        self.t1 = t1

    def setT2(self, t2):
        self.t2 = t2
        # perhaps write control sequence setting that t2 must be leq 2T1

    def setReferenceFrame(self, frameOmega):
        self.omega_rof = frameOmega

    # omega counterclockwise: rotates +x (1,0,0) into +y (0,1,0) after time pi/2
    # t1 relaxes towards 0 (lower energy) state
    def evolve(self, time):

        if (self.t1 is None): 
            d1 = 1
        else: 
            d1 = np.exp( - time * (1/self.t1))
        if (self.t2 is None):
            d2 = 1
        else: 
            d2 = np.exp( - time * (1/self.t2))
        
        self.rho = np.array([
            [ 1 - (1-self.rho[0][0])*d1, # relaxes towards 0 state over time
            self.rho[0][1]*d2 * (np.exp(1j*self.omega*time)) ], 
            [ self.rho[1][0]*d2 * (np.exp(-1j*self.omega*time)), 
            self.rho[1][1]*d1 ] ])


# checking t1,t2 functionality
q = qubitsim()
q.setStateVector([[1,0,0]])
print(q.state())

q.setT1(1)
q.setT2(2)

q.evolve(1)
print(q.state())

q.evolve(-1)
q.evolve(0.5)
q.evolve(0.5)
print(q.state())


[[0.5+0.j 0.5+0.j]
 [0.5+0.j 0.5+0.j]]
[[0.81606+0.j 0.30327+0.j]
 [0.30327+0.j 0.18394+0.j]]
[[0.81606+0.j 0.30327+0.j]
 [0.30327+0.j 0.18394+0.j]]
